References :<br>
https://www.youtube.com/watch?v=6zm9NC9uRkk <br>
https://github.com/skipgram/modern-nlp-in-python-2019/blob/master/notebooks/Modern_NLP_in_Python.ipynb

In [1]:
import os
data_directory = os.path.join('.','yelp_dataset')

businesses_filepath = os.path.join(data_directory, 'business.json')

with open(businesses_filepath) as f:
    first_business_record = f.readline() 
    
print(first_business_record)

{"business_id":"f9NumwFMBDn751xgFiRbNA","name":"The Range At Lake Norman","address":"10913 Bailey Rd","city":"Cornelius","state":"NC","postal_code":"28031","latitude":35.4627242,"longitude":-80.8526119,"stars":3.5,"review_count":36,"is_open":1,"attributes":{"BusinessAcceptsCreditCards":"True","BikeParking":"True","GoodForKids":"False","BusinessParking":"{'garage': False, 'street': False, 'validated': False, 'lot': True, 'valet': False}","ByAppointmentOnly":"False","RestaurantsPriceRange2":"3"},"categories":"Active Life, Gun\/Rifle Ranges, Guns & Ammo, Shopping","hours":{"Monday":"10:0-18:0","Tuesday":"11:0-20:0","Wednesday":"10:0-18:0","Thursday":"11:0-20:0","Friday":"11:0-20:0","Saturday":"11:0-20:0","Sunday":"13:0-18:0"}}



In [2]:
#Reviews

review_path = os.path.join(data_directory,'review.json')

with open(review_path) as f:
    first_review = f.readline()
    
print(first_review)

{"review_id":"xQY8N_XvtGbearJ5X4QryQ","user_id":"OwjRMXRC0KyPrIlcjaXeFQ","business_id":"-MhfebM0QIsKt87iDN-FNw","stars":2.0,"useful":5,"funny":0,"cool":0,"text":"As someone who has worked with many museums, I was eager to visit this gallery on my most recent trip to Las Vegas. When I saw they would be showing infamous eggs of the House of Faberge from the Virginia Museum of Fine Arts (VMFA), I knew I had to go!\n\nTucked away near the gelateria and the garden, the Gallery is pretty much hidden from view. It's what real estate agents would call \"cozy\" or \"charming\" - basically any euphemism for small.\n\nThat being said, you can still see wonderful art at a gallery of any size, so why the two *s you ask? Let me tell you:\n\n* pricing for this, while relatively inexpensive for a Las Vegas attraction, is completely over the top. For the space and the amount of art you can fit in there, it is a bit much.\n* it's not kid friendly at all. Seriously, don't bring them.\n* the security is n

In [10]:
#Extract all business ids
import json

restaurant_ids=set()

with open(businesses_filepath) as f:
    
    for business_record in f:
        business = json.loads(business_record)
        
        if not business.get('categories'):
            continue
        
        if 'Restaurants' not in business['categories']:
            continue
            

        restaurant_ids.add(business['business_id'])
        
restaurant_ids = frozenset(restaurant_ids)

print(f'{len(restaurant_ids)} Restaurants in Dataset')

63944 Restaurants in Dataset


In [7]:
#Create a cratch directory to store all reviews
scratch_path = os.path.join('.','scratch')
try:
    os.mkdir(scratch_path)
except FileExistsError:
    pass

review_txt_filepath = os.path.join(scratch_path,'review_text_all.txt')

In [15]:
#Extract all reviews snd store it in the new file created

execute = False

review_count = 0

if execute:
    
    with open(review_txt_filepath,"w") as review_txt_file:
        
        with open(review_path) as review_file:
            
            for review_json in review_file:
                review = json.loads(review_json)
                
                if review['business_id'] not in restaurant_ids:
                    continue
                    
                review_txt_file.write(review['text'].replace("\n","\\n")+"\n")
                review_count+=1
                
    print(f'Text from {review_count:,} restaurant reviews written to the new txt file.')

else:
    
    # count the reviews in the existing file
    with open(review_txt_filepath) as review_txt_file:
        for review_count, line in enumerate(review_txt_file):
            pass
        
    print(f'Text from {review_count + 1:,} restaurant reviews in the txt file.')
                
                


Text from 5,058,162 restaurant reviews in the txt file.


In [13]:
import spacy
from spacy import displacy
import pandas as pd
import itertools as it

nlp = spacy.load('en_core_web_sm')

In [16]:
#Length of each review
review_dict={}
with open(review_txt_filepath) as review_txt_file:
    review_dict = {review_count:len(line) for review_count,line in enumerate(review_txt_file)}
        
    

In [19]:
max_key = max(review_dict,key=review_dict.get)
print(max_key)

1788257


In [20]:
#Sample Review

review_num = max_key
with open(review_txt_filepath) as f:
    sample_review = list(it.islice(f, review_num, review_num+1))[0]
    sample_review = sample_review.replace('\\n', '\n')
        
print(sample_review)

WED, MARCH 27, 2013 // 12:00PM-2:00PM 
                 PRICE FOR 2: $63.98 (TR price) + $5.18 (tax) + $10.84 (tip) = $80.00
                 WAIT: 15-20mins
                 SERVER: DAWN - (4/5). She looked frazzled but always made time to check up on us. Asked me if I wanted more Peachy Tea when I finished, refilled our waters, & brought us 2 carafes when we asked for coffee.

FRI, MARCH 29, 2013 // 5:00PM-7:30PM
                 PRICE FOR 2: $99.98 (TR price) + $8.10 (tax) + $16.42 (tip) = $124.50
                 WAIT: 25-30mins
                 SERVER: IRANDA - (1/5). We had to chase her down 3x, she ignored our table completely. When we asked for coffee, she brought us 1/2 empty cups (spilled coffee all over the saucers). VINCE (4/5), not our server, was friendly, paid closer attention & cleared our table multiple times.
_________________________

Bacchanal can be overwhelming since there are a LOT of dishes to choose from. Between Bacchanal & Wicked Spoon, if you only have time 